# 🔬 Lab: Validação dos Retrievers Individuais

Este notebook demonstra **passo a passo** como funcionam os três tipos de retrieval implementados no projeto, com ênfase especial no **retriever de grafo** (entidades + embeddings).

## Objetivos
1. **TF-IDF Retriever**: Entender retrieval baseado em sobreposição lexical
2. **Dense Retriever**: Compreender embeddings semânticos (MiniLM vs BGE)
3. **Graph Retriever**: Explorar em profundidade a extração de entidades, IDF e agregação ponderada

Usaremos um mini-corpus de 5 documentos e 3 queries para validação.

## 📦 Setup: Imports e Configuração do Ambiente

Primeiro, garantimos que o Python encontra o repositório e importamos as dependências necessárias.

In [1]:
# Se estiver em um ambiente limpo, descomente conforme necessário:
# %pip install -U sentence-transformers transformers torch scikit-learn spacy scispacy

import sys
import warnings
from pathlib import Path
import numpy as np

repo_root = Path.cwd()
while repo_root != repo_root.parent and repo_root.name != "hybrid-retrieval":
    repo_root = repo_root.parent
repo_root_str = str(repo_root)
if repo_root_str not in sys.path:
    sys.path.insert(0, repo_root_str)

print("Repo root:", repo_root)

# Imports dos schemas do projeto
from src.datasets.schema import Document, Query

# Silencia warnings de modelos
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)

Repo root: /Users/thiago/Documents/GitHub/hybrid-retrieval


## 📚 Criação do Mini-Corpus

Criamos 5 documentos com temas variados para testar os diferentes comportamentos dos retrievers:
- **D1**: COVID-19 e vacinas (domínio médico, entidades: "COVID-19", "mRNA vaccines")
- **D2**: Mercados financeiros (domínio financeiro, entidades: "equity market", "US economy")
- **D3**: Graph Neural Networks (domínio técnico, entidades: "GNNs", "entity classification")
- **D4**: Nutrição (domínio saúde, entidades: "apples", "fiber", "vitamins")
- **D5**: Large Language Models (domínio ML, entidades: "MiniLM", "BGE")

E 3 queries que exploram diferentes aspectos:
- **Q1**: Busca sobre eficácia de vacinas (termos relacionados a D1)
- **Q2**: Busca sobre impacto de taxas de juros (relacionada a D2)
- **Q3**: Busca sobre embeddings de entidades e grafos (relacionada a D3 e D5)

In [2]:
docs = [
    Document(
        doc_id="D1", 
        title="COVID-19 vaccines", 
        text="Efficacy and side effects of mRNA COVID-19 vaccines in clinical trials."
    ),
    Document(
        doc_id="D2", 
        title="Financial markets overview", 
        text="Equity market volatility and interest rates in the US economy during 2023."
    ),
    Document(
        doc_id="D3", 
        title="Graph neural networks", 
        text="Using GNNs for entity classification and link prediction tasks in knowledge graphs."
    ),
    Document(
        doc_id="D4", 
        title="Nutritional benefits of apples", 
        text="Apples contain dietary fiber and vitamins, reducing cardiovascular health risks."
    ),
    Document(
        doc_id="D5", 
        title="Large language models for retrieval", 
        text="MiniLM and BGE embeddings improve semantic retrieval and reranking performance."
    ),
]

queries = [
    Query(query_id="Q1", text="Are mRNA vaccines effective against COVID?"),
    Query(query_id="Q2", text="How do interest rates impact equity volatility?"),
    Query(query_id="Q3", text="Entity embeddings and graph-based retrieval methods"),
]

print(f"✓ Corpus: {len(docs)} documentos")
print(f"✓ Queries: {len(queries)} consultas")

✓ Corpus: 5 documentos
✓ Queries: 3 consultas


---
# 🔤 Parte 1: TF-IDF Retriever (Sinal Lexical)

## O que é TF-IDF?
- **Term Frequency (TF)**: Frequência do termo no documento
- **Inverse Document Frequency (IDF)**: Penaliza termos comuns no corpus
- **Vetor TF-IDF**: Cada documento é representado como um vetor esparso de tamanho = vocabulário

## Como funciona?
1. **Fit**: Constrói vocabulário do corpus e calcula IDF para cada termo
2. **Transform**: Converte documento → vetor TF-IDF normalizado (L2)
3. **Busca**: Similaridade por inner-product (equivalente a cosseno após normalização)

## Vantagens
✓ Rápido e interpretável  
✓ Bom para matching exato de termos  

## Limitações
✗ Não captura sinônimos ou paráfrases  
✗ Sensível a variações morfológicas  

In [ ]:
from src.retrievers.tfidf_faiss import TFIDFRetriever

tfidf_r = TFIDFRetriever(
    dim=1000,              # tamanho máximo do vocabulário
    min_df=1,              # aceita termos que aparecem pelo menos 1x
    backend="sklearn",     # usa scikit-learn TfidfVectorizer
    use_faiss=True,        # IndexFlatIP para busca rápida
    artifact_dir=None,     # sem persistência (apenas para lab)
    index_name="tfidf.index",
)

print("🔧 Construindo índice TF-IDF...")
tfidf_r.build_index(docs)

print("\n🔍 Executando retrieval (top-3 por query)...")
tfidf_results = tfidf_r.retrieve(queries, k=3)

# Exibe resultados
for q in queries:
    print(f"\n📊 Query [{q.query_id}]: {q.text}")
    pairs = tfidf_results.get(q.query_id, [])
    for rank, (doc_id, score) in enumerate(pairs, 1):
        doc = next(d for d in docs if d.doc_id == doc_id)
        print(f"  {rank}. {doc_id} (score={score:.4f}) - {doc.title}")

🔧 Construindo índice TF-IDF...
2025-10-29 18:07:11 | INFO     | retriever.tfidf | [tfidf_faiss.py:66] | 🚀 Building TF-IDF Index (5 documentos)
2025-10-29 18:07:11 | INFO     | retriever.tfidf | [logging.py:199] | ⏱️  Fit TF-IDF no corpus - iniciando...
2025-10-29 18:07:11 | INFO     | tfidf.vectorizer | [logging.py:199] | ⏱️  Fit TF-IDF - iniciando...
2025-10-29 18:07:12 | INFO     | tfidf.vectorizer | [logging.py:220] | ✓ Fit TF-IDF - concluído em 1.21s
2025-10-29 18:07:12 | INFO     | tfidf.vectorizer | [tfidf_vectorizer.py:19] | ✓ TF-IDF fitted: vocab_size=60
2025-10-29 18:07:12 | INFO     | retriever.tfidf | [logging.py:220] | ✓ Fit TF-IDF no corpus - concluído em 1.21s
2025-10-29 18:07:12 | INFO     | retriever.tfidf | [logging.py:199] | ⏱️  Encoding documents (TF-IDF) - iniciando...
2025-10-29 18:07:12 | INFO     | retriever.tfidf | [logging.py:220] | ✓ Encoding documents (TF-IDF) - concluído em 1.0ms
2025-10-29 18:07:12 | INFO     | retriever.tfidf | [logging.py:199] | ⏱️  Const

: 

### 📈 Análise dos Resultados TF-IDF

**Observe**:
- Queries com **termos exatos** do corpus (ex: "mRNA", "vaccines", "COVID") ranqueiam bem documentos correspondentes
- Queries com **paráfrases** (ex: "effective" vs "efficacy") podem ter scores mais baixos
- O TF-IDF é **lexical**: não entende que "interest rates" e "taxas de juros" são sinônimos

**Exemplo**: Q1 ranquea D1 no topo, pois compartilha "mRNA", "vaccines", "COVID".

---
# 🧠 Parte 2: Dense Retriever (Embeddings Semânticos)

## O que é Dense Retrieval?
- Usa **modelos de linguagem** (ex: MiniLM, BGE) para gerar embeddings densos
- Cada documento/query → vetor contínuo de 384d (MiniLM) ou 1024d (BGE-Large)
- Similaridade captura **significado semântico**, não apenas palavras exatas

## Modelos comparados no paper
| Modelo | Dimensão | Parâmetros | Uso |
|--------|----------|------------|-----|
| **MiniLM-L6-v2** | 384 | 22M | Rápido, eficiente |
| **BGE-Large** | 1024 | 335M | Maior qualidade |

## Por que MiniLM pode superar BGE no híbrido?
O paper mostra que embeddings menores podem ter **melhor alinhamento com LLMs** durante o reranking (fenômeno chamado "FAISS Hybrid Paradox").

In [ ]:
from src.retrievers.dense_faiss import DenseFaiss

# Escolha do modelo (altere para testar)
dense_model = "sentence-transformers/all-MiniLM-L6-v2"  
# Alternativa: "BAAI/bge-large-en-v1.5"

dense_r = DenseFaiss(
    model_name=dense_model,
    device=None,              # Use "cuda:0" se tiver GPU disponível
    query_prefix="",
    doc_prefix="",
    use_faiss=True,
    artifact_dir=None,
    index_name="dense.index",
)

print(f"🔧 Construindo índice denso com {dense_model}...")
print(f"   Dimensão: {dense_r.dim}d")
dense_r.build_index(docs)

print("\n🔍 Executando retrieval (top-3 por query)...")
dense_results = dense_r.retrieve(queries, k=3)

for q in queries:
    print(f"\n📊 Query [{q.query_id}]: {q.text}")
    pairs = dense_results.get(q.query_id, [])
    for rank, (doc_id, score) in enumerate(pairs, 1):
        doc = next(d for d in docs if d.doc_id == doc_id)
        print(f"  {rank}. {doc_id} (score={score:.4f}) - {doc.title}")

/Users/thiago/Documents/GitHub/hybrid-retrieval/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🔧 Construindo índice denso com sentence-transformers/all-MiniLM-L6-v2...
   Dimensão: 384d
2025-10-29 18:07:15 | INFO     | retriever.dense | [dense_faiss.py:75] | 🚀 Building Dense Index (5 documentos)
2025-10-29 18:07:15 | INFO     | retriever.dense | [logging.py:199] | ⏱️  Encoding documents - iniciando...


### 📈 Análise dos Resultados Dense

**Observe**:
- Melhor captura de **sinônimos** e **paráfrases** (ex: "effective" ≈ "efficacy")
- Scores geralmente mais **uniformes** do que TF-IDF (espaço denso é mais suave)
- Pode ranquear documentos **tematicamente relacionados** mesmo sem overlap lexical

**Experimento**: Troque para `"BAAI/bge-large-en-v1.5"` e compare os resultados!

---
# 🕸️ Parte 3: Graph Retriever (Entidades + Embeddings)

## Visão Geral
O retriever de grafo implementa o **terceiro modo (g)** do paper:
1. **Extração de Entidades**: Identifica "nomes" importantes no texto (pessoas, locais, termos técnicos)
2. **IDF de Entidades**: Calcula importância relativa de cada entidade no corpus
3. **Embedding de Entidades**: Cada entidade → vetor denso (ex: BGE-Large 1024d)
4. **Agregação TF-IDF**: Vetor do documento = Σ [TF(e) × IDF(e) × emb(e)]
5. **Normalização L2**: Vetor final é normalizado para comparação por cosseno

## Fórmula

g(text) = L2_norm( Σ_{e ∈ entities(text)} TF(e, text) × IDF(e) × embedding(e) )


## Por que é útil?
- Captura **sinal estruturado** de entidades nomeadas
- Útil em domínios com **terminologia específica** (medicina, finanças, produtos)
- Complementa sinais denso e lexical no híbrido

## 🔍 Passo 1: Extração de Entidades

Vamos explorar **manualmente** como as entidades são extraídas de um documento exemplo antes de usar o retriever completo.

### Métodos de NER disponíveis:
1. **sciSpaCy** (recomendado para textos científicos): `en_ner_bc5cdr_md`, `en_core_sci_md`
2. **spaCy padrão**: `en_core_web_sm`, `en_core_web_md`
3. **Fallback regex** (quando spaCy não está instalado): detecta palavras capitalizadas e termos longos

### O que é extraído?
- **Entidades nomeadas**: detectadas pelo modelo NER (ex: DISEASE, CHEMICAL, ORG)
- **Noun chunks** (opcional): sintagmas nominais (ex: "mRNA vaccines", "clinical trials")

In [ ]:
# Vamos inspecionar a extração de entidades no documento D1 manualmente

doc_text = docs[0].title + " " + docs[0].text
print(f"📄 Documento D1:\n{doc_text}\n")

# Tentamos carregar spaCy (pode falhar se não instalado)
try:
    import spacy
    # Tenta sciSpaCy primeiro (melhor para textos científicos)
    try:
        nlp = spacy.load("en_ner_bc5cdr_md", disable=["tagger", "parser", "lemmatizer"])
        print("✓ Usando sciSpaCy: en_ner_bc5cdr_md")
    except:
        # Fallback para spaCy padrão
        nlp = spacy.load("en_core_web_sm", disable=["tagger", "parser", "lemmatizer"])
        print("✓ Usando spaCy: en_core_web_sm")
    
    doc = nlp(doc_text)
    
    # Extrai entidades nomeadas
    entities = []
    print("\n🏷️  Entidades Nomeadas Detectadas:")
    for ent in doc.ents:
        entities.append(ent.text.lower().strip())
        print(f"   - '{ent.text}' [{ent.label_}]")
    
    # Extrai noun chunks
    print("\n📦 Noun Chunks Detectados:")
    for chunk in doc.noun_chunks:
        chunk_text = chunk.text.lower().strip()
        if chunk_text not in entities:
            entities.append(chunk_text)
        print(f"   - '{chunk.text}'")
    
    print(f"\n✓ Total de entidades únicas: {len(set(entities))}")
    print(f"  Entidades: {sorted(set(entities))}")
    
except Exception as e:
    print(f"⚠️  spaCy não disponível: {e}")
    print("   Usando fallback regex (detecta termos capitalizados e longos)")
    
    # Fallback simples: palavras capitalizadas ou termos longos
    import re
    pattern = re.compile(r"[A-Za-z][A-Za-z0-9_\-/\.]{1,}")
    candidates = []
    for tok in pattern.findall(doc_text):
        if len(tok) >= 10 or tok[0].isupper():
            candidates.append(tok.lower())
    print(f"\n  Entidades detectadas (fallback): {sorted(set(candidates))}")

### 💡 Interpretação da Extração

**Entidades nomeadas** capturam:
- Nomes próprios: "COVID-19", "US", "MiniLM"
- Termos técnicos: "mRNA", "GNNs", "equity market"
- Conceitos importantes: "vaccines", "clinical trials"

**Noun chunks** adicionam:
- Sintagmas nominais completos: "mRNA vaccines", "interest rates"
- Contexto adicional além de palavras isoladas

**Normalização aplicada**:
- Lowercase para uniformidade
- Remoção de pontuação nas bordas
- Mínimo 2 caracteres

## 🧮 Passo 2: Cálculo do IDF de Entidades

Após extrair entidades de **todos** os documentos, calculamos o IDF:

IDF(e) = log((1 + N) / (1 + DF(e))) + 1.0


Onde:
- **N** = número total de documentos
- **DF(e)** = número de documentos que contêm a entidade `e`

**Intuição**: Entidades raras (baixo DF) recebem maior peso; entidades comuns (alto DF) são penalizadas.

In [ ]:
# Vamos calcular manualmente o IDF das entidades no nosso corpus

from collections import defaultdict
import re

print("🔢 Calculando IDF de Entidades no Corpus\n")

# Função simplificada de extração (fallback regex)
def extract_entities_simple(text):
    """Extrai entidades via regex (fallback)"""
    pattern = re.compile(r"[A-Za-z][A-Za-z0-9_\-/\.]{1,}")
    entities = []
    for tok in pattern.findall(text):
        if len(tok) >= 10 or tok[0].isupper():
            entities.append(tok.lower().strip(".,;:()[]{}"))
    return [e for e in entities if len(e) >= 2]

# Conta DF (document frequency) de cada entidade
df = defaultdict(int)
N = len(docs)

for doc in docs:
    text = (doc.title or "") + " " + (doc.text or "")
    entities = set(extract_entities_simple(text))
    for e in entities:
        df[e] += 1

# Calcula IDF
entity_idf = {}
for e, count in df.items():
    idf_value = np.log((1 + N) / (1 + count)) + 1.0
    entity_idf[e] = idf_value

# Ordena por IDF decrescente
sorted_entities = sorted(entity_idf.items(), key=lambda x: x[1], reverse=True)

print(f"📊 Top 10 Entidades por IDF (mais raras = maior IDF):\n")
for e, idf in sorted_entities[:10]:
    print(f"   {e:<20} | DF={df[e]:>2} | IDF={idf:.4f}")

print(f"\n✓ Total de entidades únicas no corpus: {len(entity_idf)}")

### 📈 Interpretação do IDF

**Entidades com IDF alto** (raras):
- Aparecem em poucos documentos
- São mais **discriminativas** (ajudam a diferenciar documentos)
- Exemplo: "cardiovascular" (só em D4), "gnns" (só em D3)

**Entidades com IDF baixo** (comuns):
- Aparecem em muitos documentos
- São menos úteis para distinguir conteúdo
- Exemplo: termos genéricos que aparecem em múltiplos contextos

**Por que isso importa?** 
O IDF pondera a contribuição de cada entidade no vetor final, dando mais peso a termos discriminativos.

## 🎯 Passo 3: Embedding de Entidades

Cada entidade é transformada em um **vetor denso** usando um modelo HF (ex: BGE-Large).

**Exemplo**: 
- Entidade: `"mRNA vaccines"`
- Embedding: vetor de 1024 dimensões (se BGE-Large)

**Por que embeddings de entidades?**
- Captura **similaridade semântica entre entidades** (ex: "mRNA" ≈ "RNA-based")
- Permite que entidades relacionadas contribuam de forma similar ao vetor do documento

In [ ]:
# Vamos ver como uma entidade é transformada em embedding

from src.encoders.encoders import HFSemanticEncoder, l2norm

print("🧬 Gerando Embeddings de Entidades\n")

# Carrega encoder (mesmo usado no retriever de grafo)
entity_encoder = HFSemanticEncoder(
    model_name="BAAI/bge-large-en-v1.5",  # BGE-Large (1024d)
    device=None,
)

print(f"✓ Modelo: {entity_encoder.model_name}")
print(f"✓ Dimensão: {entity_encoder.dim}d\n")

# Embute algumas entidades exemplo
sample_entities = ["mrna vaccines", "covid-19", "equity market", "gnns"]

print("📊 Embeddings de Entidades (primeiras 5 dimensões):\n")
embeddings = {}
for ent in sample_entities:
    emb = l2norm(entity_encoder.encode_text(ent, is_query=False))
    embeddings[ent] = emb
    norm = np.linalg.norm(emb)
    print(f"   {ent:<20} → [{emb[:5]}...] || norm={norm:.4f}")

print("\n✓ Embeddings gerados e normalizados (L2 norm ≈ 1.0)")

### 🔗 Similaridade entre Entidades

Vamos calcular a **similaridade de cosseno** entre pares de entidades para verificar se o modelo captura relações semânticas.

In [ ]:
# Calcula similaridade de cosseno entre pares de entidades

def cosine_sim(v1, v2):
    return float(np.dot(v1, v2))  # Já normalizados (L2), então dot = cosseno

print("🔗 Similaridade entre Entidades (cosseno):\n")

pairs = [
    ("mrna vaccines", "covid-19"),      # relacionadas (domínio médico)
    ("equity market", "gnns"),          # não relacionadas (finanças vs ML)
    ("mrna vaccines", "equity market"), # não relacionadas (medicina vs finanças)
]

for e1, e2 in pairs:
    if e1 in embeddings and e2 in embeddings:
        sim = cosine_sim(embeddings[e1], embeddings[e2])
        print(f"   '{e1}' ↔ '{e2}': {sim:.4f}")

print("\n💡 Entidades semanticamente relacionadas têm similaridade maior!")

## 🧩 Passo 4: Agregação TF-IDF de Entidades

Agora combinamos tudo para gerar o **vetor do documento**:

g(doc) = L2_norm( Σ_{e ∈ doc} TF(e, doc) × IDF(e) × embedding(e) )


Onde:
- **TF(e, doc)** = frequência da entidade no documento
- **IDF(e)** = inverse document frequency (calculado no passo 2)
- **embedding(e)** = vetor denso da entidade (calculado no passo 3)

**Algoritmo**:
1. Extrair entidades do documento
2. Contar TF de cada entidade
3. Para cada entidade: peso = TF × IDF
4. Somar: vetor_final = Σ (peso × embedding)
5. Normalizar L2

In [ ]:
# Vamos calcular manualmente o vetor de grafo para o documento D1

doc_d1 = docs[0]
text_d1 = (doc_d1.title or "") + " " + (doc_d1.text or "")
print(f"📄 Documento D1:\n{text_d1}\n")

# 1. Extrai entidades
entities_d1 = extract_entities_simple(text_d1)
print(f"🏷️  Entidades extraídas: {entities_d1}\n")

# 2. Conta TF
from collections import Counter
tf_d1 = Counter(entities_d1)
print(f"🔢 Term Frequency (TF):")
for e, count in tf_d1.items():
    print(f"   {e}: {count}")

# 3. Calcula vetor agregado (manual)
print(f"\n🧮 Agregação TF-IDF × Embeddings:")
vec_d1 = np.zeros(entity_encoder.dim, dtype=np.float32)

for e, tf in tf_d1.items():
    if e in entity_idf:  # entidade conhecida no corpus
        idf = entity_idf[e]
        weight = tf * idf
        
        # Gera embedding da entidade (ou usa cache)
        if e not in embeddings:
            embeddings[e] = l2norm(entity_encoder.encode_text(e, is_query=False))
        emb = embeddings[e]
        
        vec_d1 += weight * emb
        print(f"   {e:<20} | TF={tf} | IDF={idf:.4f} | weight={weight:.4f}")

# 4. Normaliza L2
vec_d1 = l2norm(vec_d1)
print(f"\n✓ Vetor final (primeiras 5 dims): {vec_d1[:5]}")
print(f"✓ Norma L2: {np.linalg.norm(vec_d1):.4f}")

### 💡 Interpretação da Agregação

**Contribuição de cada entidade**:
- Entidades **frequentes** no documento (alto TF) contribuem mais
- Entidades **raras** no corpus (alto IDF) têm maior peso relativo
- Embedding captura o **significado semântico** da entidade

**Resultado final**:
- Vetor denso de 1024d (se BGE-Large) normalizado
- Representa o documento como uma **composição ponderada de suas entidades**
- Permite comparação por cosseno com queries que também passam pelo mesmo processo

## 🚀 Retriever de Grafo Completo

Agora que entendemos o passo a passo, vamos usar o **GraphRetriever** oficial que automatiza todo esse pipeline:

1. **Fit**: Extrai entidades de todos os documentos e calcula IDF
2. **Build**: Gera embeddings e vetores agregados, indexa no FAISS
3. **Retrieve**: Para cada query, gera vetor e busca top-K por cosseno

In [ ]:
from src.retrievers.graph_faiss import GraphRetriever

graph_r = GraphRetriever(
    graph_model_name="BAAI/bge-large-en-v1.5",  # BGE-Large para embeddings de entidades
    device=None,                                # "cuda:0" para GPU
    ner_backend="scispacy",                     # ou "spacy" / "none" (fallback)
    ner_model=None,                             # auto-detecta modelo disponível
    ner_use_noun_chunks=True,                   # inclui noun chunks
    ner_batch_size=64,
    ner_n_process=1,
    ner_allowed_labels=None,                    # aceita todas as labels NER
    min_df=1,                                   # min document frequency
    use_faiss=True,
    artifact_dir=None,
    index_name="graph.index",
    entity_artifact_dir=None,                   # sem cache neste lab
    entity_force_rebuild=False,
)

print("🔧 Construindo índice de grafo...")
print(f"   Modelo: {graph_r.vec.encoder.model_name}")
print(f"   Dimensão: {graph_r.vec.dim}d")
graph_r.build_index(docs)

print("\n🔍 Executando retrieval (top-3 por query)...")
graph_results = graph_r.retrieve(queries, k=3)

for q in queries:
    print(f"\n📊 Query [{q.query_id}]: {q.text}")
    pairs = graph_results.get(q.query_id, [])
    for rank, (doc_id, score) in enumerate(pairs, 1):
        doc = next(d for d in docs if d.doc_id == doc_id)
        print(f"  {rank}. {doc_id} (score={score:.4f}) - {doc.title}")

### 📈 Análise dos Resultados Graph

**Observe**:
- Queries com **entidades nomeadas fortes** (ex: "mRNA", "COVID", "entity embeddings") devem ranquear bem documentos relacionados
- O retriever de grafo é especialmente forte quando:
  - Documento e query compartilham **entidades raras** (alto IDF)
  - Entidades têm **alta similaridade semântica** nos embeddings
- Pode capturar relações que TF-IDF perde (paráfrases de entidades) e que Dense não enfatiza (importância de termos específicos)

**Exemplo esperado**: 
- Q3 ("Entity embeddings and graph-based retrieval") deve ranquear bem D3 (GNNs, entity classification) e D5 (embeddings para retrieval)

---
# 📊 Comparação Lado a Lado dos Três Retrievers

Vamos consolidar os resultados dos três métodos para facilitar a análise comparativa.

In [ ]:
import pandas as pd

def format_results(results, name):
    """Formata resultados em DataFrame para visualização"""
    rows = []
    for q in queries:
        pairs = results.get(q.query_id, [])[:3]
        for rank, (doc_id, score) in enumerate(pairs, 1):
            doc = next(d for d in docs if d.doc_id == doc_id)
            rows.append({
                'Query': q.query_id,
                'Retriever': name,
                'Rank': rank,
                'Doc': doc_id,
                'Score': f"{score:.4f}",
                'Title': doc.title[:40]
            })
    return pd.DataFrame(rows)

# Concatena resultados
df_tfidf = format_results(tfidf_results, "TF-IDF")
df_dense = format_results(dense_results, "Dense")
df_graph = format_results(graph_results, "Graph")

df_all = pd.concat([df_tfidf, df_dense, df_graph], ignore_index=True)

print("📊 Comparação dos Retrievers (Top-3 por Query):\n")
for qid in ["Q1", "Q2", "Q3"]:
    print(f"\n{'='*80}")
    print(f"Query {qid}: {next(q.text for q in queries if q.query_id == qid)}")
    print('='*80)
    subset = df_all[df_all['Query'] == qid]
    print(subset[['Retriever', 'Rank', 'Doc', 'Score', 'Title']].to_string(index=False))

### 🔍 Análise Comparativa

| Retriever | Força | Fraqueza |
|-----------|-------|----------|
| **TF-IDF** | ✓ Match exato de termos<br>✓ Rápido e interpretável | ✗ Não captura sinônimos<br>✗ Sensível a vocabulário |
| **Dense** | ✓ Similaridade semântica<br>✓ Generalização | ✗ Pode perder signal lexical específico<br>✗ Menos interpretável |
| **Graph** | ✓ Captura entidades importantes<br>✓ Ponderação por IDF | ✗ Depende da qualidade do NER<br>✗ Mais lento (extração + embedding) |

**Conclusão**: Cada retriever captura um **sinal complementar**. O retriever **híbrido** combina os três para obter o melhor de todos.

---
# 🔬 Exploração Adicional: Dimensões e Normalização

Vamos verificar as dimensões e normas L2 dos vetores gerados por cada retriever para confirmar que estão corretamente normalizados.

In [ ]:
# Verifica dimensões e normalização

print("📐 Dimensões e Normalização dos Vetores\n")

# TF-IDF
from src.vectorizers.tfidf_vectorizer import TFIDFVectorizer
tf_vec = TFIDFVectorizer(dim=1000, min_df=1, backend="sklearn")
tf_vec.fit_corpus([(d.title or "") + " " + (d.text or "") for d in docs])
v_tf = tf_vec.encode_text(queries[0].text)
print(f"TF-IDF:")
print(f"   Dim: {v_tf.shape[0]:>4}d | L2 norm: {np.linalg.norm(v_tf):.6f} | Sparsity: {np.sum(v_tf == 0) / len(v_tf) * 100:.1f}%")

# Dense (MiniLM)
from src.vectorizers.dense_vectorizer import DenseVectorizer
dv = DenseVectorizer(model_name="sentence-transformers/all-MiniLM-L6-v2", device=None)
v_d = dv.encode_query(queries[0].text)
print(f"\nDense (MiniLM):")
print(f"   Dim: {v_d.shape[0]:>4}d | L2 norm: {np.linalg.norm(v_d):.6f} | Sparsity: {np.sum(v_d == 0) / len(v_d) * 100:.1f}%")

# Graph
from src.vectorizers.graph_vectorizer import GraphVectorizer
gv = GraphVectorizer(
    graph_model_name="BAAI/bge-large-en-v1.5", 
    device=None, 
    ner_backend="scispacy", 
    min_df=1
)
gv.fit_corpus([(d.title or "") + " " + (d.text or "") for d in docs])
v_g = gv.encode_text(queries[0].text)
print(f"\nGraph (BGE-Large):")
print(f"   Dim: {v_g.shape[0]:>4}d | L2 norm: {np.linalg.norm(v_g):.6f} | Sparsity: {np.sum(v_g == 0) / len(v_g) * 100:.1f}%")

print("\n✓ Todos os vetores estão normalizados (L2 norm ≈ 1.0)")
print("✓ TF-IDF é esparso; Dense e Graph são densos")

### 📊 Interpretação das Dimensões

**TF-IDF** (1000d):
- Dimensão = tamanho do vocabulário (limitado por `max_features`)
- **Altamente esparso**: maioria das dimensões é zero (termos não presentes)
- Sparsity típica: 95-99%

**Dense MiniLM** (384d):
- Dimensão fixa definida pelo modelo
- **Totalmente denso**: todas as dimensões têm valores não-zero
- Menor que BGE, mas ainda captura semântica eficazmente

**Graph BGE-Large** (1024d):
- Dimensão = dimensão do encoder de entidades
- **Denso**: composição ponderada de embeddings de entidades
- Maior dimensão → maior capacidade representacional

**Normalização L2**: Todos os vetores têm norma ≈ 1.0, permitindo comparação direta por inner-product (equivalente a cosseno).